<a href="https://colab.research.google.com/github/prbsh9/Suspicious-Activity-Detection-from-videos/blob/master/RNNImageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and syncing with drive

In [0]:
from google.colab import drive
drive.mount('/drive')
import os
path = '/drive/My Drive/suspicious_activity/content/frameEasy'
os.chdir(path)
os.getcwd()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


'/drive/My Drive/suspicious_activity/content/frameEasy'

In [0]:
%matplotlib inline
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Dense, Flatten, Dropout
from keras.callbacks import CSVLogger
from keras.layers import Activation
from keras.layers import CuDNNLSTM
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint



# RNN part?

In [0]:
os.listdir()

['train',
 'models',
 'valid',
 'predict2.jpg',
 'predict4.jpg',
 'predict5.jpg',
 'predict',
 'Road Accident Live Video in KTM Nepal CCTV Footage-360p.webm',
 'export.pkl',
 '#cctv #gang #fight-360p.webm',
 'image_classifier',
 'test',
 '.ipynb_checkpoints',
 'final.csv',
 '3.csv',
 '4.csv',
 '2.csv',
 '1.csv',
 'hey.npy',
 'data',
 'valid_abnorm.npy',
 'valid_normal123.npy',
 'train_normal1.npy',
 'train_normal1111.npy',
 'Normal_Videos_006_x264.mp4']

(4000, 8, 8, 2048)

## Make train files

In [0]:
import numpy as np
# train_normal = np.load('rnnData/train_normal.npy')
# train_abnormal = np.load('rnnData/train_abnormal.npy')


In [0]:
# train_normal = train_normal[:6000]
# np.save('rnnData/train_normal.npy', train_normal)

In [0]:
# train_abnormal = train_abnormal[:4000]
# np.save('rnnData/train_abnormal.npy', train_abnormal)


In [0]:
# train_normal_y = np.ones(6000)
# train_abnormal_y = np.zeros(4000)


In [0]:
# train_x = np.concatenate((train_normal, train_abnormal))

In [0]:
# np.save('data/train_data.npy', train_x)

In [0]:
# train_y = np.concatenate((train_normal_y, train_abnormal_y))
# np.save('data/train_labels.npy', train_y)


## Make test or valid files

In [0]:
valid_normal = np.load('data/valid_normal123.npy')
valid_abnormal = np.load('data/valid_abnorm.npy')

# valid_abnormal = np.load('rnnData/train_abnormal.npy')

In [0]:
valid_normal = valid_normal[:2000]
valid_abnormal = valid_abnormal[:2000]


In [0]:
valid_data = np.concatenate((valid_normal, valid_abnormal))
np.save('data/valid_data.npy',valid_data )
np.save('rnnData/valid_data.npy',valid_data )


In [0]:
valid_normal_y = np.ones(2000)
valid_abnormal_y = np.zeros(2000)
valid_labels = np.concatenate((valid_normal_y, valid_abnormal_y))
np.save('data/valid_labels.npy',valid_labels )
np.save('rnnData/valid_labels.npy',valid_labels )


# What about training it boy?

In [0]:
train_data = np.load('data/train_data.npy') 
train_labels = np.load('data/train_labels.npy') 

# valid_data = np.load('x_valid.npy')
# valid_labels = np.load('y_valid.npy')

In [0]:
train_data = train_data/255.0
valid_data = valid_data/255.0

In [0]:
train_data = train_data.reshape(train_data.shape[0],
                     train_data.shape[1] * train_data.shape[2],
                     train_data.shape[3])
valid_data = valid_data.reshape(valid_data.shape[0],
                     valid_data.shape[1] * valid_data.shape[2],
                     valid_data.shape[3])

In [0]:
model = Sequential()
model.add(CuDNNLSTM(2024, input_shape=(train_data.shape[1:]), return_sequences=True ))
model.add(Dropout(0.6))

model.add(CuDNNLSTM(1028))
model.add(Dropout(0.5))

model.add(Dense(2024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.6))

model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
sgd = SGD(lr=0.00005, decay = 1e-5, momentum=0.9, nesterov=True)
sgd = Adam(lr=0.00005, decay = 1e-5)

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

callbacks = [ EarlyStopping(monitor='val_loss', patience=10, verbose=0), ModelCheckpoint('video_1_LSTM_1_1024.h5', monitor='val_loss', save_best_only=True, verbose=0) ]
nb_epoch = 500
batch_size = 64

model.fit(train_data,train_labels,validation_data=(valid_data,valid_labels),batch_size=batch_size,nb_epoch=nb_epoch,callbacks=callbacks,shuffle=True,verbose=1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Train on 10000 samples, validate on 4000 samples
Epoch 1/500
10000/10000 [==============================] - 44s 4ms/step - loss: 0.7019 - acc: 0.5599 - val_loss: 0.6969 - val_acc: 0.5000
Epoch 2/500
10000/10000 [==============================] - 42s 4ms/step - loss: 0.6832 - acc: 0.5865 - val_loss: 0.7026 - val_acc: 0.5000
Epoch 3/500
10000/10000 [==============================] - 42s 4ms/step - loss: 0.6778 - acc: 0.5975 - val_loss: 0.7035 - val_acc: 0.5000
Epoch 4/500
10000/10000 [==============================] - 42s 4ms/step - loss: 0.6765 - acc: 0.5989 - val_loss: 0.7088 - val_acc: 0.5000
Epoch 5/500
10000/10000 [==============================] - 42s 4ms/step - loss: 0.6750 - acc: 0.5991 - val_loss: 0.7096 - val_acc: 0.5000
Epoch 6/500
10000/10000 [==============================] - 42s 4ms/step - loss: 0.6740 - acc: 0.5997 - val_loss: 0.7107 - val_acc: 0.5000
Epoch 7/500
10000/10000 [==============================] - 42s 4ms/step - loss: 0.6740 - acc: 0.5994 - val_loss: 0.7109 - v

In [0]:
# 
# frames_num=9
# count = 0
# joint_transfer=[]
# for i in range(int(len(feature('predict5.jpg'))/frames_num)):
#     inc = count+frames_num
#     joint_transfer.append([transfer_values[count:inc],labels_train[count]])
#     count =inc

In [0]:
# LSTM

chunk_size = 2048
n_chunks = 9
nb_classes = 2
model = Sequential()
model.add(CuDNNLSTM(2048, input_shape=(n_chunks, chunk_size)))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# for filee in os.listdir('data/train/Abnorm'):
#   train_npy = np.load('data/train/Abnorm/'+filee)
#   frames_num=9
#   count = 0
#   joint_transfer=[]
#   for i in range(int(len(train_npy))/frames_num):
#       inc = count+frames_num
#       joint_transfer.append([transfer_values[count:inc],labels_train[count]])
#       count =inc

In [0]:
# import shutil
# shutil.copyfile('train_normal1.npy','data/train_normal1.npy' )

'data/train_normal1.npy'

In [0]:
# emp = np.empty((1, 8, 8, 2048))
# filepth = 'data/train/Normal/'
# for filee in os.listdir(filepth):
#   c = np.load(filepth+filee)
#   emp = np.concatenate((emp, c))
#   print( filee, emp.shape[0] )
#   if 10000 < emp.shape[0] < 10010:
#     print('Saved!!!!!!!!!!!!!!!!!!')
#     np.save('train_normal1.npy', emp)
   
#   if 15000 < emp.shape[0] < 15010:
#     print('Saved!!!!!!!!!!!!!!!!!!')
#     np.save('train_normal2.npy', emp)
#     break


rmal_Videos164_x264-features15.npy 10
rmal_Videos164_x264-features16.npy 19
rmal_Videos164_x264-features17.npy 28
rmal_Videos164_x264-features18.npy 37
rmal_Videos164_x264-features19.npy 46
rmal_Videos164_x264-features20.npy 55
rmal_Videos164_x264-features21.npy 64
rmal_Videos164_x264-features22.npy 73
rmal_Videos164_x264-features23.npy 82
rmal_Videos164_x264-features24.npy 91
rmal_Videos164_x264-features25.npy 100
rmal_Videos164_x264-features26.npy 109
rmal_Videos164_x264-features27.npy 118
rmal_Videos164_x264-features28.npy 127
rmal_Videos164_x264-features29.npy 136
rmal_Videos164_x264-features30.npy 145
rmal_Videos164_x264-features31.npy 154
rmal_Videos164_x264-features32.npy 163
rmal_Videos164_x264-features33.npy 172
rmal_Videos164_x264-features34.npy 181
rmal_Videos164_x264-features35.npy 190
rmal_Videos164_x264-features36.npy 199
rmal_Videos164_x264-features37.npy 208
rmal_Videos164_x264-features38.npy 217
rmal_Videos071_x264-features1.npy 226
rmal_Videos071_x264-features2.npy 23

KeyboardInterrupt: ignored

In [0]:
train_Ynormal = np.zeros(10009)
valid_ynormal = np.zeros(3998)

train_yabnorm = np.zeros()
valid_yabnorm = np.zeros(4398)


(10009,)

In [0]:
import numpy as np
# arsa is train_Normal
# Train_Normal is of 10009 
# Valid Normal is of 3998
# valid_abnorm is 4398
# del arsasss

NameError: ignored

In [0]:
import numpy as np
dummy = np.load('data/valid/Normal/rmal_Videos_453_x264-features1.npy')
# arsasss = np.load('train_normal1111.npy')
# arsasss = np.load('valid_normal123.npy')                                                              

# arsasss = np.load('data/train_abnormal1.npy')                                                                                         
# arsasss = np.load('valid_abnorm.npy')

        
                                                        
dummy.shape

(9, 8, 8, 2048)

In [0]:
# if performance becomes bad.. think of this as one way of improving it.
# filepth = 'data/train/Normal/'
# for filee in os.listdir(filepth):
#   c = np.load(filepth+filee)
#   if c.shape[0] < 9:
#     print(filee)
#     os.remove(filepth+filee)

In [0]:
data =[]
target=[]
epoch = 1500
batchS = 100
for i in joint_transfer:
    data.append(i)
    target.append(np.array(i[1]))
model.fit(data, target, epochs=epoch, batch_size=batchS, verbose=1)

IndexError: ignored

In [0]:

  
# c.shape[0]

In [0]:
for a in os.listdir('data/valid/Abnorm'):
  npy = np.load(a)
  

  

In [0]:
def train(data_type='features', seq_length=40, model='lstm', saved_model=None,
          class_limit=None, image_shape=None,
          load_to_memory=False, batch_size=32, nb_epoch=100):
  

  # Get the data and process it.
      data = DataSet(seq_length=seq_length, class_limit=class_limit)


  # Get samples per epoch.
  # Multiply by 0.7 to attempt to guess how much of data.data is the train set.
  steps_per_epoch = (len(data.data) * 0.7) // batch_size

  if load_to_memory:
      # Get data.
      X, y = data.get_all_sequences_in_memory('train', data_type)
      X_test, y_test = data.get_all_sequences_in_memory('test', data_type)
  else:
      # Get generators.
      generator = data.frame_generator(batch_size, 'train', data_type)
      val_generator = data.frame_generator(batch_size, 'test', data_type)

  # Get the model.
  rm = LSTM()

      # Use fit generator.
  rm.model.fit_generator(
      generator=generator,
      steps_per_epoch=steps_per_epoch,
      epochs=nb_epoch,
      verbose=1,
      callbacks=[tb, early_stopper, csv_logger, checkpointer],
      validation_data=val_generator,
      validation_steps=40,
      workers=4)


In [0]:
# Testing
probabilities = model.predict_generator(test_generator, TEST_SIZE)
for index, probability in enumerate(probabilities):
    image_path = test_data_dir + "/" +test_generator.filenames[index]
    img = mpimg.imread(image_path)
    plt.imshow(img)
    if probability > 0.5:
        plt.title("%.2f" % (probability[0]*100) + "% dog")
    else:
        plt.title("%.2f" % ((1-probability[0])*100) + "% cat")
    plt.show()